# Workflow and web application for annotation of NCBI BioProject transcriptome data 

Roberto Vera Alvarez$^{1}$, Newton Medeiros Vidal$^{1}$, Gina A Garzón-Martínez$^{2}$, Luz S Barrero$^{2}$, Richa Agarwala$^{3}$, David Landsman$^{1}$ and Leonardo Mariño-Ramírez$^{1,*}$

$^{1}$Computational Biology Branch, National Center for Biotechnology Information, National Library of Medicine, National Institutes of Health, Bethesda, Maryland, USA

$^{2}$Colombian Corporation for Agricultural Research (CORPOICA), Bogota, Colombia

$^{3}$Information Engineering Branch, National Center for Biotechnology Information, National Library of Medicine, National Institutes of Health, Bethesda, Maryland, USA

$^{*}$Corresponding author: Email: marino@ncbi.nlm.nih.gov Address: Building 38A, Room 6S614-M, 8600 Rockville Pike MSC 6075. Bethesda, MD 20894-6075 Tel: 1-301-402-3708

## Abstract

Experimental data, specifically data coming from next-generation DNA sequencing technologies is growing exponentially due to the latest improvements of experimental devices. In response to that, large central resources, such as those of the National Center for Biotechnology Information (NCBI), are continually adapting their computational infrastructure to accommodate this data influx. New specialized databases have been created like Transcriptome Shotgun Assembly Sequence Database (TSA) and Sequence Read Archive (SRA) aiding the creation of centralized repositories. Although these databases are in a constant development, they do not include automatic pipelines to increase the annotation of the deposited data. Therefore, third party applications are required to achieve that aim. Here we present an automatic workflow and web application for the annotation of NCBI transcriptome data that is poorly annotated. A case study is presented for the species Physalis peruviana with data generated from the BioProject with ID 67621. The workflow includes the creation of secondary data like NGS read alignments and BLAST results which are available through the web application. The workflow is based on freely available bioinformatics tools and a set of in-house developed scripts. The interactive web application provides a search engine and different browse utilities. Additionally, graphical views of the transcript alignments are available through SeqViewer, a friendly embedded tool developed by NCBI for viewing biological sequence data. The web application is tightly integrated to the others NCBI web applications and tools extending the possibility of data processing and interconnectivity.


## Notebook

This is a Jupyter Notebook which illustrate the workflow used for the transcript annotation process. This Github project can be clone and executed in a compatible environment.

## Requirements

### Basic programs

1. Python3
2. virtualenv3
3. C++

### Bioinformatics programs

These programs have to be installed and included in the user PATH variable.

1. EDirect (https://www.ncbi.nlm.nih.gov/books/NBK179288/)
2. SRA Toolkit (https://www.ncbi.nlm.nih.gov/books/NBK158900/)
3. Bowtie 2 (http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml)
4. SAMTools (http://www.htslib.org/)
5. BLAST (http://blast.ncbi.nlm.nih.gov/Blast.cgi)

## Instalation of the Notebook

Execute this commands in a BASH terminal just the first time to get the notebook dependencies installed.

```
$ git clone 
$ cd trans-annot-notebook

# Compiling the myBlast2GO C++ tool
$ cd src/myblast2go/
$ make
$ cd ../../

# Creating a virtual environment
$ virtualenv3 venv
$ source venv/bin/activate
$ pip install -r requirements/jupyter.txt
$ chmod a+x ./bin/*
```

Execute this script to start the jupyter server.

```
$ ./bin/runNotebook.sh
```

After this open a web browser with this URL: http://localhost:9096

Click on docs and then, click on Notebook.ipynb to open the notebook



In [12]:
# Global imports
import os
import re
import csv
import sqlite3
import xmltodict
from Bio import Entrez
from Bio import SeqIO

In [2]:
# Email to be used by Entrez
Entrez.email = "veraalva@ncbi.nlm.nih.gov"

In [3]:
# Environment variables
os.environ['WORKDIR'] = os.path.abspath('../')
os.environ['CONFIG'] = os.environ['WORKDIR'] + '/config'
os.environ['DATA'] = os.environ['WORKDIR'] + '/data'
os.environ['BIN'] = os.environ['WORKDIR'] + '/bin'
os.environ['RESULTS'] = os.environ['WORKDIR'] + '/results'
os.environ['DOC'] = os.environ['WORKDIR'] + '/docs'
os.environ['SRC'] = os.environ['WORKDIR'] + '/src'
os.environ['SQLITEDBSCHEMA'] = os.environ['CONFIG'] + '/sqlite_database_schema.sql'

In [4]:
# BioProject, SRA Id and SQLite database
os.environ['BIOPROJECT'] = 'PRJNA67621'
os.environ['SRA'] = 'SRP005904'
os.environ['SQLITEDB'] = os.environ['RESULTS'] + '/' + os.environ['BIOPROJECT'] + '.sqlite'

In [76]:
%%bash

# Initialize SQLite database
if [ ! -e "${SQLITEDB}" ]
then
    cat $SQLITEDBSCHEMA | sqlite3 $SQLITEDB
fi

memory


In [19]:
%%bash

# Downloading the SRR data using fastq-dump
cd $DATA
if [ ! -e "${SRA}" ]
then
    mkdir ${SRA}
    sh $BIN/download_SRA.sh -b ${BIOPROJECT} -w ./${SRA}
else
    echo "The data is ready"
fi

The data is ready


In [24]:
%%bash

# Download the UniProt ID mapping to link Gi with Protein ID
cd $DATA
if [ ! -e "idmapping_selected.tab" ]
then
    wget -o wget.log ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz
    gunzip idmapping_selected.tab.gz
fi

In [5]:
# Get the SRA IDs related to the BioProject from NCBI
handle = Entrez.esearch(db="sra", term=os.environ['BIOPROJECT'], retmax=50)
sra_id = Entrez.read(handle)
handle.close()

In [6]:
# Retrieving the bioproject from NCBI
handle = Entrez.esearch(db="bioproject", term=os.environ['BIOPROJECT'])
bioproject_ids = Entrez.read(handle)
handle.close()
os.environ['BIOPROJECTID'] = bioproject_ids['IdList'][0]
handle = Entrez.efetch(db="bioproject", id=os.environ['BIOPROJECTID'])
bioproject = xmltodict.parse(handle.read())
handle.close()

In [77]:
# Inserting the BioProject data into the SQLite DB
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
c.execute("INSERT INTO 'bioprojects' ('id', 'path') VALUES (" + os.environ['BIOPROJECTID'] + ", '" + os.environ['WORKDIR'] + "')")
conn.commit()
conn.close()

In [29]:
%%bash

# Downloading the BioProject transcripts 
cd $DATA
if [ ! -e "bioproject-${BIOPROJECT}.fsa" ]
then
    idfetch -t5 -c1 -q "${BIOPROJECTID}[BioProject]" -dn > bioproject-${BIOPROJECT}.fsa
fi

In [44]:
# Retirve each experiment related to the SRA IDs from NCBI
sra = {}
for id in sra_id['IdList']:
    if id != '297619':
        handle = Entrez.efetch(db="sra", id=id, rettype="full", retmode="full")
        sra[id] = xmltodict.parse(handle.read())
        handle.close()    

In [78]:
# Insert the experiments and runs in the database
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
for key in sra:
    c.execute("INSERT INTO 'experiments' ('accession', 'title', 'bioproject_id') VALUES ('" + key + "', '" + sra[key]['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['EXPERIMENT']['TITLE'] + "', '" + os.environ['BIOPROJECTID'] + "')")
    conn.commit()
    c.execute("SELECT id FROM 'experiments' where accession = '" + key + "'")
    experiment_id = c.fetchone()
    if type(sra[key]['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['RUN_SET']['RUN']) is list: 
        for run in sra[key]['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['RUN_SET']['RUN']:
            c.execute("INSERT INTO 'runs' ('accession', 'experiment_id') VALUES ('" + run['@accession'] + "', " + str(experiment_id[0]) + ")")
            conn.commit()  
    else:
        c.execute("INSERT INTO 'runs' ('accession', 'experiment_id') VALUES ('" + sra[key]['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['RUN_SET']['RUN']['@accession'] + "', " + str(experiment_id[0]) + ")")
        conn.commit()        
conn.commit()
conn.close()

In [107]:
# Parsinf the FASTA file and insert the IDs into the database
# A folder for each transcript is created into the results folder
# Bowtie2 and blast are executed

if not os.path.isdir(os.environ['RESULTS'] + '/transcripts'):
    os.makedirs(os.environ['RESULTS'] + '/transcripts')
    
os.chdir(os.environ['RESULTS'] + '/transcripts')

rpoint = re.compile('[\|\.]')
rPipe = re.compile('\|')

conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
with open(os.environ['DATA'] + '/bioproject-' + os.environ['BIOPROJECT'] + '.fsa', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        desc = rPipe.split(record.description)
        gi = ""
        acc = ""
        title = desc[len(desc) - 1].strip()
        for j in range(0, len(desc)):
            if desc[j] == "gi":
                gi = desc[j + 1]
            if desc[j] == "gb":
                acc = rpoint.split(desc[j + 1])

        if gi == "" or acc == "" or title == "":
            print("Error parsing record: " + record.id + " " + record.description)
            break

        if not os.path.exists(acc[0]):
            os.makedirs(acc[0])

        os.chdir(acc[0])
        
        if not os.path.exists(acc[0] + ".fasta"):  
            toInsert = (acc[0], acc[1], gi, title, len(record.seq), os.path.abspath('./'), os.environ['BIOPROJECTID'])
            c.execute("INSERT INTO nucleotides ('accession', 'version', 'gi', 'title', 'len', 'path','bioproject_id')  VALUES (? ,?, ?, ?, ?, ?, ?)", toInsert)
            output = open(acc[0] + ".fasta", "w")
            SeqIO.FastaIO.FastaWriter(output, wrap=80).write_file([record])
            output.close()
        
        if not os.path.exists(acc[0] + "_sorted.bam"):
            os.environ['ID'] = acc[0]
            !sh $BIN/processTranscript.sh -i $ID -s $DATA/$SRA
        
        os.chdir("../")


conn.commit()
conn.close()

In [116]:
%%bash

# Proecessing Blast results to be inserted into the database
cd $RESULTS
sh $BIN/processBlastResults.sh -t ./transcripts -b $BIN/buildLine.py

Retrieving taxonomies, iteration 


In [20]:
# Loading the taxonomies to the SQLite DB
os.chdir(os.environ['RESULTS'])
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
with open('taxonomies_to_load.tsv') as handle:
    rows = csv.reader(handle, delimiter='\t')
    c.executemany("INSERT INTO taxonomies (id, name, division) VALUES (?, ?, ?);", rows)
conn.commit()
conn.close()

In [21]:
# Loading the proteins to the SQLite DB
os.chdir(os.environ['RESULTS'])
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
with open('proteins_to_load.tsv') as handle:
    rows = csv.reader(handle, delimiter='\t')
    c.executemany("INSERT INTO proteins (taxonomy_id, gi, accession, title) VALUES (?, ?, ?, ?);", rows)
conn.commit()
conn.close()

In [22]:
# Loading the Blast results to the SQLite DB
os.chdir(os.environ['RESULTS'])
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
with open('blastx_to_load.tsv') as handle:
    rows = csv.reader(handle, delimiter='\t')
    c.executemany("INSERT INTO blastx_temp (qacc, sgi, slen, qstart, qend, sstart, send, evalue, bitscore, score, length, pident, nident, mismatch, positive, gapopen, gaps, ppos, qcovs, qcovhsp) VALUES (?, ?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?,?, ?, ?, ?,?, ?, ?, ?);", rows)
c.execute("insert into blastx (slen,qstart,qend,sstart,send,evalue,bitscore,score,length,pident,nident,mismatch,positive,gapopen,gaps,ppos,qcovs,qcovhsp,protein_id, nucleotide_id) select bt.slen,bt.qstart, bt.qend, bt.sstart, bt.send, bt.evalue, bt.bitscore, bt.score, bt.length, bt.pident, bt.nident, bt.mismatch, bt.positive, bt.gapopen, bt.gaps, bt.ppos, bt.qcovs, bt.qcovhsp, p.id, n.id from blastx_temp bt inner join nucleotides n on n.accession = bt.qacc inner join proteins p on bt.sgi = p.gi")    
c.execute("delete from blastx_temp")
conn.commit()
conn.close()

In [25]:
%%bash

# Running our Blast to GO annotation program
cd $RESULTS
if [ ! -e "blastGI2GO.tsv" ]
then
    !$BIN/myBlast2GO -d $DATA/idmapping_selected.tab -g 2 -i ./transcripts/ -l 9 -c 50 -o blastGI2GO.tsv -e blastGI2GO.err -r -s _blastx.tsv
fi    

In [28]:
# Loading the GI2GO results to the SQLite DB
os.chdir(os.environ['RESULTS'])
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
with open('blastGI2GO.tsv') as handle:
    rows = csv.reader(handle, delimiter='\t')
    c.executemany("INSERT INTO gi2go (gi, go) VALUES (?, ?);", rows)
conn.commit()
conn.close()

In [29]:
# Loading the GO data from a previusly created tsv file from JBioWH to the SQLite DB
os.chdir(os.environ['DATA'])
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
with open('go_JBioWH.tsv') as handle:
    rows = csv.reader(handle, delimiter='\t')
    c.executemany("INSERT INTO go (accession, name, description, name_space) VALUES (?, ?, ?, ?);", rows)
conn.commit()
conn.close()

In [31]:
# Creating links between nucleotides and GO terms
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
c.execute("insert OR IGNORE into nucleotides_go (nucleotides_id, go_id) select n.id,go.id from nucleotides n inner join blastx b on n.id = b.nucleotide_id inner join proteins p on p.id = b.protein_id inner join gi2go g on g.gi = p.gi inner join go on go.accession = g.go;")
conn.commit()
conn.close()

In [32]:
# Creates the GO statistics for the 20 more relevant terms by GO name space
conn = sqlite3.connect(os.environ['SQLITEDB'])
c = conn.cursor()
c.execute("insert into go_statistics (go_id,transcripts) select g.id,count(n.nucleotides_id) as c from go g inner join nucleotides_go n on n.go_id = g.id where g.name_space = 'biological_process' group by g.id order by c desc limit 20 ;")
c.execute("insert into go_statistics (go_id,transcripts) select g.id,count(n.nucleotides_id) as c from go g inner join nucleotides_go n on n.go_id = g.id where g.name_space = 'molecular_function' group by g.id order by c desc limit 20 ;")
c.execute("insert into go_statistics (go_id,transcripts) select g.id,count(n.nucleotides_id) as c from go g inner join nucleotides_go n on n.go_id = g.id where g.name_space = 'cellular_component' group by g.id order by c desc limit 20 ;")
conn.commit()
conn.close()